In [58]:
## Dependencies
import requests
from requests.exceptions import HTTPError
import json
from config import ClientID
from config import Clientsecret
from urllib.parse import urlencode
from pandas.io.json import json_normalize  
import pandas as pd

import sys, os
import re
import glob
import numpy as np
from pprint import pprint



## ORCID (orcid-python) package is outdated
## https://github.com/scholrly/orcid-python
## pip list -v can be used to list packages' install locations
## C:\Users\keg827\AppData\Local\Continuum\anaconda3\Lib\site-packages

## Take a look at Python 3.0 Wiki Built-in Changes section, where it is stated:
## Removed dict.iteritems(), dict.iterkeys(), and dict.itervalues().
## Instead: use dict.items(), dict.keys(), and dict.values() respectively.

# import orcid
# from inspect import getmembers, isfunction
# print(getmembers(orcid, isfunction))
# help(orcid)

## Python-Orcid package also does not work: 
## https://github.com/ORCID/python-orcid

## Pathway for reading data from ORCID Member API 

In this pathway you will use the /read-public endpoint. Do not confuse this with the /read-limited endpoint, as that endpoint is meant for reading data from a specific user record and requires the 3-leg authentication (i.e. exchanging the code for the user's token) where the user must authenticate which gives you (or your web app) permission to read the data. 

The /read-public end point is sufficient if you want to search and return data from the ORCID database from your university, for example. 

In [ ]:
## Step 1: Use the /oath/token endpoint to receive the token needed for using the /read-pubic endpoint

## Set up header and parameters for post request

headers = {
    "accept": "application/json",
    "method": "post"
}

data = {
    "client_id": ClientID,
    "client_secret": Clientsecret,
    "grant_type" : "client_credentials",
    "scope": "/read-public"
}


## Create try/catch loop for request

## production API = https://api.orcid.org
## sandbox API = https://sandbox.orcid.org/oauth/toke

for url in ["https://sandbox.orcid.org/oauth/token"]:
    try: 
        ## Make the API post request 
        response = requests.post(url, headers = headers, data = data) 
        
        ## Print responses
        print(response.status_code)
        print(response.url)
        print(response.text)
        print(type(response.text))
        response_string = response.text
        
        ## If the response was successful, no Exception will be raised
        response.raise_for_status()
    
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
   
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    
    else:
        print('Success!')
        

## Resources
## https://info.orcid.org/documentation/api-tutorials/api-tutorial-read-data-on-a-record/#easy-faq-2536

In [135]:
 ## Step 2: Use the access token and call the /read-public endpoint

## Save the token as a variable
json_token = json.loads(response_string)
print(json_token["access_token"])
final_token = json_token["access_token"]


## Save the query items as variables
ringgold_id = "3270" 
grid_id = "grid.16753.36" 
email_domain = "@northwestern.edu" 
organization_name = "Northwestern University"
organization_name2 = "Northwestern"

## Save the items in a list that you'd like returned

# fieldList = ["orcid", "email", "given-name","family-name", "given-and-family-names", "credit-name",
#              "other-name","current-institution-affiliation-name","past-institution-affiliation-name"]


## Set up query for request

query_dict = {
     "ringgold-org-id":ringgold_id,
     "OR grid-org-id":grid_id, 
     "OR email": email_domain,
     "OR affiliation-org-name": organization_name,
     "affiliation-org-name": organization_name2
}

query_str = urlencode(query_dict)
#print(query_str)

#search_url = f"https://api.sandbox.orcid.org/v3.0/expanded-search/?q={query_str}" 

## production API = https://api.orcid.org
## sandbox API = https://sandbox.orcid.org/oauth/toke
#print(search_url)

## Set up headers and parameters for request

headers = {
    "Accept": "application/json",
    "Authorization": f'Bearer{final_token}' 
}


# parameters = {
#     #"access_token": final_token,
#    # "q" : query_str,
#     "field": ",".join(fieldList)
    
# }

#search_url = "https://api.sandbox.orcid.org/search/?"

search_response_list = []

def get_request(query_str, headers, search_response_list):
    
    #search_response_list = []
    start = 0
    rows = 1000

    while True: 
        
        try:
                
                ## Create URL for request
                print("------")
                search_url = f"https://api.sandbox.orcid.org/v3.0/expanded-search/?q={query_str}&start={start}&rows={rows}" 
                print("Requesting", search_url)
                
                ## Make the API post request 
                search_response = requests.get(search_url, headers = headers,  timeout=30) # params = parameters,
                #print(type(search_response))
                json_data = search_response.json()
                #print(json_data["expanded-result"][0]["orcid-id"])
                #print(type(json_data))
                
                ## Print responses
                print("Search Status", search_response.status_code)
                # print(search_response.url)
                # print(search_response.text)
                # print(type(search_response.text))
                
                
                ## If the response was successful, no Exception will be raised
                search_response.raise_for_status()
                
                ## Need something that will give us an error, or else it will keep working
                print(json_data["expanded-result"][0]["orcid-id"]) 
                
               
               
                ## If we did find more ORCID data, add them to the list and move on to next offset
                search_response_list.extend(json_data["expanded-result"])
                
                
                
        except HTTPError as http_err:
                print(f'HTTP error occurred: {http_err}')  # Python 3.6
                break

        except Exception as err:
                print(f'Other error occurred: {err}')  # Python 3.6
                print(search_response.url)
                break

        else:
                print('Success!')
        
        start = start + 1000

    return search_response_list


get_request(query_str, headers, search_response_list)

print(len(search_response_list))
print(type(search_response_list))

## Notes
## Please note the Public API is limited to 10,000 results. 
## Using the Member API (with a Member API token) does not limit the number of results.

## Resources
## https://info.orcid.org/faq/how-do-i-find-orcid-record-holders-at-my-institution/
## https://info.orcid.org/documentation/api-tutorials/api-tutorial-read-data-on-a-record/#easy-faq-2361
## https://info.orcid.org/documentation/api-tutorials/api-tutorial-searching-the-orcid-registry/
## https://realpython.com/python-api/#request-and-response
## http://www.compciv.org/guides/python/how-tos/creating-proper-url-query-strings/
## https://stackoverflow.com/questions/17788445/constructing-requests-with-url-query-string-in-python
## Using paginated APIs (4 ways): https://www.youtube.com/watch?v=4Fdyft-ky0w



f3e2ef7f-0d36-4bc6-b4aa-64fd59f0a75d
------
Requesting https://api.sandbox.orcid.org/v3.0/expanded-search/?q=ringgold-org-id=3270&OR+grid-org-id=grid.16753.36&OR+email=%40northwestern.edu&OR+affiliation-org-name=Northwestern+University&affiliation-org-name=Northwestern&start=0&rows=1000
Search Status 200
0000-0002-0564-3270
Success!
------
Requesting https://api.sandbox.orcid.org/v3.0/expanded-search/?q=ringgold-org-id=3270&OR+grid-org-id=grid.16753.36&OR+email=%40northwestern.edu&OR+affiliation-org-name=Northwestern+University&affiliation-org-name=Northwestern&start=1000&rows=1000
Search Status 200
0000-0002-4672-494X
Success!
------
Requesting https://api.sandbox.orcid.org/v3.0/expanded-search/?q=ringgold-org-id=3270&OR+grid-org-id=grid.16753.36&OR+email=%40northwestern.edu&OR+affiliation-org-name=Northwestern+University&affiliation-org-name=Northwestern&start=2000&rows=1000
Search Status 200
0000-0002-9257-0304
Success!
------
Requesting https://api.sandbox.orcid.org/v3.0/expanded-s

Search Status 200
0000-0001-5716-0302
Success!
------
Requesting https://api.sandbox.orcid.org/v3.0/expanded-search/?q=ringgold-org-id=3270&OR+grid-org-id=grid.16753.36&OR+email=%40northwestern.edu&OR+affiliation-org-name=Northwestern+University&affiliation-org-name=Northwestern&start=28000&rows=1000
Search Status 200
0000-0001-5852-7401
Success!
------
Requesting https://api.sandbox.orcid.org/v3.0/expanded-search/?q=ringgold-org-id=3270&OR+grid-org-id=grid.16753.36&OR+email=%40northwestern.edu&OR+affiliation-org-name=Northwestern+University&affiliation-org-name=Northwestern&start=29000&rows=1000
Search Status 200
0000-0001-6287-051X
Success!
------
Requesting https://api.sandbox.orcid.org/v3.0/expanded-search/?q=ringgold-org-id=3270&OR+grid-org-id=grid.16753.36&OR+email=%40northwestern.edu&OR+affiliation-org-name=Northwestern+University&affiliation-org-name=Northwestern&start=30000&rows=1000
Search Status 200
0000-0003-1771-8126
Success!
------
Requesting https://api.sandbox.orcid.org

[{'orcid-id': '0000-0002-0564-3270',
  'given-names': 'ALIREZA',
  'family-names': 'DOOSTAN',
  'credit-name': None,
  'other-name': [],
  'email': [],
  'institution-name': ['University of Colorado Boulder']},
 {'orcid-id': '0000-0002-3270-2838',
  'given-names': 'Priya',
  'family-names': 'Joshi',
  'credit-name': None,
  'other-name': [],
  'email': ['priya.joshi454@bmc.org', 'pjoshi454@bu.edu'],
  'institution-name': []},
 {'orcid-id': '0000-0001-9962-7513',
  'given-names': 'Researcher',
  'family-names': 'Test',
  'credit-name': None,
  'other-name': [],
  'email': [],
  'institution-name': ['Mc Gill University',
   'My College',
   'My University',
   'The Place to Study',
   'University ABCD',
   'University EFGH']},
 {'orcid-id': '0000-0002-3270-5131',
  'given-names': 'Carl',
  'family-names': "O'Hara",
  'credit-name': None,
  'other-name': [],
  'email': ['hara461@bu.edu'],
  'institution-name': []},
 {'orcid-id': '0000-0002-3270-2950',
  'given-names': 'Albert',
  'family-

In [141]:
print(len(search_response_list))
print(type(search_response_list))
print(search_response_list[1])
print(type(search_response_list[1]))

36399
<class 'list'>
{'orcid-id': '0000-0002-3270-2838', 'given-names': 'Priya', 'family-names': 'Joshi', 'credit-name': None, 'other-name': [], 'email': ['priya.joshi454@bmc.org', 'pjoshi454@bu.edu'], 'institution-name': []}
<class 'dict'>


In [ ]:
## Step 3: Convert response, which isa list of dictionaries into pandas dataframe

orcid_results_df = pd.json_normalize(search_response_list, record_path=["expanded-result"])
orcid_results_df.head()


<class 'str'>
<class 'dict'>


,orcid-id,given-names,family-names,credit-name,other-name,email,institution-name
0,0000-0002-0564-3270,ALIREZA,DOOSTAN,None,[],[],[University of Colorado Boulder]
1,0000-0002-3270-2838,Priya,Joshi,None,[],"[priya.joshi454@bmc.org, pjoshi454@bu.edu]",[]
2,0000-0001-9962-7513,Researcher,Test,None,[],[],"[Mc Gill University, My College, My University..."
3,0000-0002-3270-5131,Carl,O'Hara,None,[],[hara461@bu.edu],[]
4,0000-0002-3270-2950,Albert,Einstein,None,[],[],[]


In [91]:
## Step 4: Save dataframe to a CSV

with open(r"orcid_results_df.csv", 'w', encoding='utf-8') as file:
    orcid_results_df.to_csv(file, line_terminator='\n', index=True)
    file.close()

## Pathway for editing an ORCID record using the ORCID Member API

### Pathway code note yet complete

Note, this pathway is similar to if a user were to give permission to Scopus to update their record. So it doesn't really translate well into a Jupyter Notebook becuase we're not a web application asking a user to give us permission. More so, this pathway is included here so you can see how it works. 

In [ ]:
## Step 1: Authenticate with ORCID to receive a token

## https://info.orcid.org/documentation/integration-guide/getting-started-with-your-orcid-integration/#easy-faq-2569

## Set up URL and parameters
url = "https://sandbox.orcid.org/oauth/authorize?"

parameters = {
    "client_id": ClientID,
    "client_secret": Clientsecret,
    "response_type" : "code",
    "scope": "read-limited",
    "redirect_uri" : "https://api.sandbox.orcid.org"
    
}

             
## Make the API request 

response = requests.get(url, params = parameters) #headers = headers,

## Print responses
print(response.status_code)
print(response.url)
# print(response.text)

## Click on the URL and authenticate with your sandbox ORCID username and password. 
## Don't yet have sandbox ORCID username and password? 
## username must be: XXXXX@mailinator.com
## Need to reset your sandbox ORCID password?
## You can go to https://www.mailinator.com/ and enter your username in the textbox 
## This will take you to your public mailbox and where youwill find the password reset email

## Resources
## https://info.orcid.org/faq/how-does-3-legged-oauth-work/

In [ ]:
## Step 2: Exchange the code received for a token. 
## The code expires upon use but the token is good for multiple uses and expires after 20 years

URL=https://sandbox.orcid.org/oauth/token
  HEADER: Accept: application/json
  HEADER: Content-Type: application/x-www-form-urlencoded
  METHOD: POST
  DATA: 
    client_id=[Your client ID]
    client_secret=[Your client secret]
    grant_type=authorization_code
    code=Six-digit code
    redirect_uri=[Your landing page]
    
## Resources
## https://info.orcid.org/faq/how-does-3-legged-oauth-work/


In [ ]:
## Step 3: Use the token in API requests you make to read or update that record.



In [ ]:
# https://pub.sandbox.orcid.org/v3.0/search/?q=family-name:Einstein+AND+keyword:Relativity&start=0&rows=10
#             https://pub.sandbox.orcid.org/v3.0/search/?q=email:*@orcid.org

# Sandbox: https://api.sandbox.orcid.org

# Production: https://api.orcid.org

# Method: GET
#   Content-type: text/csv
#   Authorization type: Bearer
#   Access token: [Stored access token]
#   URL:  https://api.orcid.org/v3.0/csv-search/?q=ringgold-org-id:385488&fl=orcid,given-names,family-name,current-institution-affiliation-name,'